In [1]:
import pandas as pd
import numpy as np
import math
from bridge_specs import box
from Cross import calc_centroid as cg
from irc6_2007 import ka



In [2]:
def loader(unit_wt, length, width, height, type):
    if type == 'r':
        load = unit_wt*length*width*height
    elif type == 't':
        load = unit_wt*length*width*height*0.5
    
    return load
def z(base, clr, width, x, type):
    """base = width of base along with taper
       clr = overhang part from where centroid is calculated
        width = width of object whose lever arm is required
        object = shape of object
    """
    y = []
    if type == 'r':
        y.append(cg('rectangle',[width,0],[x,0]))
        return (-(clr+base/2)+y[0][0])
    y = []
    if type == 't1':
        y.append(cg('triangle_2',[width,0],[x,0]))
        return (-(clr+base/2)+y[0][0])
    y = []
    if type == 't2':
        y.append(cg('triangle_1',[width,0],[x,0]))
        return (-(clr+base/2)+y[0][0])

Abutment details

In [3]:
bearing_ht = 0.3 #assumed including pedestal
abut = {
    'total_ht' : 7.0,
    'wc' : 0.075,
    'backw_ht' : box['pillar_hei']+bearing_ht+0.075, #including wearing cource
    'cap':0.5,
    'stem_ht': 7-box['pillar_hei']+bearing_ht+0.075-0.5,
    'bw_width':0.3,
    'edge_to_bearing_c':0.6,
    'bw_to_bearing_c': 0.6,
    'cap_width':0.3+0.6+0.6,
    'stem_width':1.25,
    'be_taperwidth':0.10,
    'width_of_abut':8.4,
    'ttl_stembase_width':1.25+0.10,
    'approach_slab_dep':0.3,
    'approach_slab_len':3.5,
    'haunch':{'stem_haunch':[0.2,0.25],'approach_slab_haunch_rec':[0.25,0.25], 'approach_slab_haunch_tri':[0.25,0.25]} #list of width and height.
}

Live Loads, Dead loads and Pedestrian loads

In [4]:
#dummy loads
dl = 3000 #dead loads
ll = 1000 #live loads including impact
pdl = 200 #pedestrian loads
sdl=250 #Surface Loads

Material properties

In [5]:
fck = 30 #grade of concrete = M30
fy = 500 #grade of steel = Fe500
unitwt_conc = 25 #unit weight of concrete = 25KN/m^3
unitwt_bfill = 20 #unit weight of backfill = 20KN/m^3

Self weight loads of abutment

In [6]:
# comp = ['stem_rect','stem_tri','cap','backwall','stem_haunch','approach_slab_haunch']
loads = [loader(unitwt_conc,abut['stem_ht'],abut['width_of_abut'],abut['stem_width'],'r'),
         loader(unitwt_conc,abut['stem_ht'],abut['width_of_abut'],abut['be_taperwidth'],'t'),
         loader(unitwt_conc,abut['cap'],abut['width_of_abut'],abut['cap_width'],'r'),
         loader(unitwt_conc,abut['backw_ht'],abut['width_of_abut'],abut['bw_width'],'r'),
         loader(unitwt_conc,abut['haunch']['stem_haunch'][0],abut['width_of_abut'],abut['haunch']['stem_haunch'][1],'r'),
         loader(unitwt_conc,abut['haunch']['approach_slab_haunch_rec'][0],abut['width_of_abut'],abut['haunch']['approach_slab_haunch_rec'][1],'r'),
          loader(unitwt_conc,abut['haunch']['approach_slab_haunch_tri'][0],abut['width_of_abut'],abut['haunch']['approach_slab_haunch_tri'][1],'t')]
lever_arm = [round(z(abut['ttl_stembase_width'],abut['haunch']['stem_haunch'][0],abut['stem_width'],abut['haunch']['stem_haunch'][0],'r'),5),
            round(z(abut['ttl_stembase_width'],abut['haunch']['stem_haunch'][0],abut['be_taperwidth'],abut['haunch']['stem_haunch'][0]+abut['stem_width'],'t1'),5),
            round(z(abut['ttl_stembase_width'],abut['haunch']['stem_haunch'][0],abut['cap_width'],0,'r'),5),
            round(z(abut['ttl_stembase_width'],abut['haunch']['stem_haunch'][0],abut['bw_width'],0,'r'),5),
            round(z(abut['ttl_stembase_width'],abut['haunch']['stem_haunch'][0],abut['haunch']['stem_haunch'][0],0,'t2'),5),
            round(z(abut['ttl_stembase_width'],abut['haunch']['stem_haunch'][0],-abut['haunch']['approach_slab_haunch_rec'][0],0,'r'),5),
            round(z(abut['ttl_stembase_width'],abut['haunch']['stem_haunch'][0],-abut['haunch']['approach_slab_haunch_tri'][0],0,'t2'),5)]
moments = []
for index,i in enumerate(loads):
    moments.append(loads[index]*lever_arm[index])
A1 = pd.DataFrame({"Load":loads,"Lever arm":lever_arm,"Moment":moments},index = ['Stem(rectangle)','Stem(triangle)','Cap','Backwall','Stem haunch','Approach slab haunch(rec)','Approach slab haunch(tri)'])
A1.loc[len(A1.index)] = [A1['Load'].sum(),None,A1['Moment'].sum()]
display(A1.rename(index={7:'Total'}))

,Load,Lever arm,Moment
Stem(rectangle),964.6875,-0.05000,-48.234375
Stem(triangle),38.5875,0.60833,23.473934
Cap,157.5000,-0.12500,-19.687500
Backwall,225.2250,-0.72500,-163.288125
Stem haunch,10.5000,-0.74167,-7.787535
Approach slab haunch(rec),13.1250,-1.00000,-13.125000
Approach slab haunch(tri),6.5625,-1.04167,-6.835959
Total,1416.1875,NaN,-235.484560


Loads from superstructure

In [7]:
load = [dl,ll+pdl,sdl]
arm = [round(z(abut['ttl_stembase_width'],abut['haunch']['stem_haunch'][0],abut['edge_to_bearing_c']+abut['bw_to_bearing_c'],abut['bw_width'],'r'),5),
      round(z(abut['ttl_stembase_width'],abut['haunch']['stem_haunch'][0],abut['edge_to_bearing_c']+abut['bw_to_bearing_c'],abut['bw_width'],'r'),5),
      round(z(abut['ttl_stembase_width'],abut['haunch']['stem_haunch'][0],abut['edge_to_bearing_c']+abut['bw_to_bearing_c'],abut['bw_width'],'r'),5)]
moment = []
for index,i in enumerate(load):
    moment.append(load[index]*arm[index])
A2 = pd.DataFrame({'Load':load, 'Lever arm':arm,'Moment':moment}, index = ['Dead load','Live load','Surface load'])
display(A2)

,Load,Lever arm,Moment
Dead load,3000,0.025,75.00
Live load,1200,0.025,30.00
Surface load,250,0.025,6.25


Earth Pressure as per clause 214 of IRC:6

In [8]:
phi = 33 #Angle of internal friction of soil
alpha = 0 #Angle which earth face of the wal makes with the vertical of the wall
beta = 0 #Slope of earth fill
delta = 2/3*phi if 2/3*phi<22.5 else 22.5 #Angle of friction between earth and earth fill
h = abut['total_ht']-abut['approach_slab_dep'] #Height of earthfill


In [23]:
ep = unitwt_bfill*h*ka(alpha, beta, delta, phi) #Earth pressure in KN/m^2.
force = 0.5*ep*abut['width_of_abut']*h #Force due to earth pressure in KN.
forc = [force*math.cos(math.radians(delta)),force*math.cos(math.radians(delta)),force*math.sin(math.radians(delta))] #Horizontal and vertical components.
larm = [0.42*h, 0.33*h, -(abut['stem_width']+abut['be_taperwidth'])/2 ] #0.42 times height of wall above base for dry soil and 0.33 for wet.
momen = []
for index,i in enumerate(forc):
    momen.append(forc[index]*larm[index])
A3 = pd.DataFrame({'Force(KN)':forc, 'Lever arm(m)':larm,'Moment(KNm)':momen}, index = ['Earth pressure Hz(dry)','Earth pressure Hz(wet)','Earth pressure Ver'])
display(A3)

,Force(KN),Lever arm(m),Moment(KNm)
Earth pressure Hz(dry),924.601823,2.814,2601.829531
Earth pressure Hz(wet),924.601823,2.211,2044.294631
Earth pressure Ver,373.563385,-0.675,-252.155285
